Main articles being used as reference:
https://www.thepythoncode.com/article/speech-recognition-using-huggingface-transformers-in-python

https://github.com/facebookresearch/fairseq/blob/main/examples/wav2vec/README.md

http://www.openslr.org/12

https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/

https://pypi.org/project/noisereduce/

https://jmvalin.ca/demo/rnnoise/


In [1]:
# Imports 
from transformers import *
import torch
import soundfile as sf
# import librosa
import os
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from os import listdir, walk
from os.path import isfile, join
from pathlib import PurePath
from pydub import AudioSegment
import shutil
from jiwer import wer
import accelerate



c:\Users\jared\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Preprocessor and model weights
model_name = "facebook/wav2vec2-base-960h" # 360MB
# model_name = "facebook/wav2vec2-large-960h-lv60-self" # 1.18GB

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# processor = Wav2Vec2Processor.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)
# model = Wav2Vec2ForCTC.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)
model.to(device)

loading configuration file preprocessor_config.json from cache at C:\Users\jared/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

loading configuration file config.json from cache at C:\Users\jared/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)

In [3]:
# looking at named parameters of wav2vec2 (not useful to finding hyperparameters)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

wav2vec2.masked_spec_embed tensor([0.8475, 0.0920, 0.8377, 0.0642, 0.6735, 0.0862, 0.9728, 0.4774, 0.9194,
        0.3482, 0.5368, 0.6919, 0.1829, 0.5737, 0.8986, 0.3248, 0.6055, 0.9204,
        0.4930, 0.5015, 0.0038, 0.0222, 0.5244, 0.4307, 0.7880, 0.0723, 0.0973,
        0.9286, 0.5168, 0.1508, 0.5739, 0.4565, 0.9462, 0.0323, 0.5537, 0.2949,
        0.0635, 0.1646, 0.3321, 0.6549, 0.5891, 0.7201, 0.6318, 0.6376, 0.2521,
        0.2019, 0.7892, 0.6033, 0.8614, 0.2045, 0.6355, 0.2623, 0.2478, 0.6985,
        0.0127, 0.4256, 0.0473, 0.9381, 0.3212, 0.5627, 0.7106, 0.0828, 0.2468,
        0.8450, 0.8794, 0.4603, 0.6232, 0.5479, 0.9279, 0.3435, 0.6323, 0.1148,
        0.2580, 0.8713, 0.2942, 0.9274, 0.4536, 0.9262, 0.4930, 0.9242, 0.5782,
        0.1792, 0.0616, 0.5549, 0.2769, 0.2502, 0.7139, 0.7658, 0.4191, 0.9125,
        0.1170, 0.1937, 0.2817, 0.8075, 0.1716, 0.9103, 0.2302, 0.3133, 0.4688,
        0.0853, 0.7807, 0.6988, 0.6757, 0.5227, 0.2312, 0.9730, 0.9359, 0.5538,
        0.465

In [12]:
# directory file path parameters
# root_dir = "C:/Users/jared/Downloads/test-clean/LibriSpeech/test-clean/"
new_root_dir = "../data/LibriSpeech/test-clean-wav/"
data_path = "../data/LibriSpeech/smol/"


In [5]:
# audio_url = "https://github.com/x4nth055/pythoncode-tutorials/raw/master/machine-learning/speech-recognition/16-122828-0002.wav"
# audio_url = "https://github.com/x4nth055/pythoncode-tutorials/raw/master/machine-learning/speech-recognition/30-4447-0004.wav"
# audio_url = "https://github.com/x4nth055/pythoncode-tutorials/raw/master/machine-learning/speech-recognition/7601-291468-0006.wav"
audio_path = "../data/raw_wav/clean.wav"

In [6]:
# define the prediction function which takes in the file path to a wav file and outputs the predicted words

def predict (model, device, path): 
    # preprocess audio & reduce noise
    rate, data = wavfile.read(path)
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate, stationary=False, prop_decrease=0.9)
    reduced_file_path = "../data/reduced/test.wav"
    wavfile.write(reduced_file_path, rate, reduced_noise)

    # prep audio 

    # load our wav file
    speech, sr = torchaudio.load(reduced_file_path)
    # print(speech.shape)
    speech = torch.mean(speech, dim=0, keepdim=True)
    speech = speech.squeeze()
    sr, speech.shape
    # print(speech.shape)

    # resample from whatever the audio sampling rate to 16000
    resampler = torchaudio.transforms.Resample(sr, 16000)
    # print(type(resampler))
    speech = resampler(speech)
    speech.to(device)

    
    # tokenize our wav
    input_values = processor(speech, return_tensors="pt", sampling_rate=16000)["input_values"].to(device)
    # input_values.shape

    # perform inference
    logits = model(input_values)["logits"]

    # use argmax to get the predicted IDs
    predicted_ids = torch.argmax(logits, dim=-1)

    # decode the IDs to text
    transcription = processor.decode(predicted_ids[0])
    return transcription.upper()


In [7]:
# testing predict
test_path = "C:\\Users\\jared\\Documents\\GitHub\\Subtitle-AI\\data\\raw_wav\\assets_fish.wav"
print(predict(model, device, test_path))

I KNOW THE HUMAN BEING AND FISHED AND CO EXIST PEACEFULLY


In [8]:
# testing function on LibriSpeech testing data


# evaluation function which will test the model on the testing data
def eval(model, device, dir):
    average_word_error_rate = 0
    num_sents = 0
    # dfs approach to read into each of the directories
    for (dirpath, dirnames, filenames) in walk(dir):
        new_dir = dirpath.replace(new_root_dir, "")
        # case where we have reached directory of audio files
        if(dirnames == []):
            # print("currently at: " + dirpath)
            # print(filenames)
            # print()

            # get the answer key for the words (transcript)
            trans_path = ""
            for fname in filenames:
                if "txt" in fname:
                    trans = fname
            # print("transcript file location is: " + trans)
            
            # from the transcript file, iterate over each line
            trans_lines = open(dirpath + "/" + trans, 'r').readlines()
            for line in trans_lines:
                words = line.split(" ", 1)
                # words[0] is the audio file name
                # call the model function on the audio file
                res = predict(model, device, dirpath + "/" + words[0] + ".wav")
                # words[1:] are the actual transcript (correct owrds)
                # print("words[0]: " + words[0])
                # print("words[1]: " + words[1])
                # print("res: " + str(res))
                # compare the two sentences
                word_error_rate = wer(words[1], res)
                average_word_error_rate += word_error_rate
                num_sents += 1
                # print("word error rate: " + str(word_error_rate))
        # else:
        #     for dname in dirnames:
        #         if not os.path.exists(new_root_dir + new_dir + "/" + dname):
        #             os.mkdir(new_root_dir + new_dir + "/" + dname)
        #     print("dirnames: " + str(dirnames))
    return (average_word_error_rate/num_sents)
print("done!")



done!


In [9]:
eval(model, device, data_path)

0.03921664633950245

In [10]:
import pandas as pd

In [14]:
# evaluation

import pandas as pd


attention_dropout=[0.1, 0.2, 0.3]
hidden_dropout=[0.1, 0.2, 0.3]
feat_proj_dropout=[0.0, 0.1, 0.2]
mask_time_prob=[0.05, 0.4, 0.6]

# attention_dropout=[0.1, 0.11, 0.12, 0.099, 0.098]
# hidden_dropout=[0.1, 0.11, 0.12, 0.099, 0.098]
# feat_proj_dropout=[0.0, 0.01, 0.02, 0.03, 0.04, 0.05]
# mask_time_prob=[0.05, 0.04, 0.045, 0.055, 0.06]

n_trials=len(attention_dropout)*len(hidden_dropout)*len(feat_proj_dropout)*len(mask_time_prob) 
current_n = 1
index = 1
print(n_trials)
 
 
df = pd.DataFrame(columns = ['attention_dropout','hidden_droupout','feat_proj_dropout','mask_time_prob',"WER"])
model_name = "facebook/wav2vec2-base-960h" 

for a in attention_dropout:
    for b in hidden_dropout:
        for c in feat_proj_dropout:
            for d in mask_time_prob:
                    print(a,b,c,d)
                    print('Currently working on ', current_n, ' of the ', n_trials,' total trials')
                    print('Progress: ',(current_n/n_trials)*100, '%')
                    configuration = Wav2Vec2Config(
                        attention_dropout=a,
                        hidden_dropout=b,
                        feat_proj_dropout=c,
                        mask_time_prob=d,
                        layerdrop=1,
                        ctc_loss_reduction="mean", 
                        vocab_size=len(processor.tokenizer))
                    # model = Wav2Vec2Model(configuration)
                    model = Wav2Vec2ForCTC .from_pretrained(model_name, config = configuration)
                    model.to(device)
                    score = eval(model, device, data_path)

                    df.loc[index]=[a,b,c,d,score]
                    index = index+1
                    current_n = current_n+1
                    print(current_n)
df



loading weights file pytorch_model.bin from cache at C:\Users\jared/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\pytorch_model.bin


81
0.1 0.1 0.0 0.05
Currently working on  1  of the  81  total trials
Progress:  1.2345679012345678 %


All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading weights file pytorch_model.bin from cache at C:\Users\jared/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\pytorch_model.bin


2
0.1 0.1 0.0 0.4
Currently working on  2  of the  81  total trials
Progress:  2.4691358024691357 %


All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading weights file pytorch_model.bin from cache at C:\Users\jared/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\pytorch_model.bin


...

All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading weights file pytorch_model.bin from cache at C:\Users\jared/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\pytorch_model.bin


81
0.3 0.3 0.2 0.6
Currently working on  81  of the  81  total trials
Progress:  100.0 %


All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


82


,attention_dropout,hidden_droupout,feat_proj_dropout,mask_time_prob,WER
1,0.1,0.1,0.0,0.05,0.039217
2,0.1,0.1,0.0,0.40,0.039217
3,0.1,0.1,0.0,0.60,0.039217
4,0.1,0.1,0.1,0.05,0.039217
5,0.1,0.1,0.1,0.40,0.039217
...,...,...,...,...,...
77,0.3,0.3,0.1,0.40,0.039217
78,0.3,0.3,0.1,0.60,0.039217
79,0.3,0.3,0.2,0.05,0.039217
80,0.3,0.3,0.2,0.40,0.039217


In [15]:
df.sort_values(by = ["WER"])

,attention_dropout,hidden_droupout,feat_proj_dropout,mask_time_prob,WER
1,0.1,0.1,0.0,0.05,0.039217
58,0.3,0.1,0.1,0.05,0.039217
57,0.3,0.1,0.0,0.60,0.039217
56,0.3,0.1,0.0,0.40,0.039217
55,0.3,0.1,0.0,0.05,0.039217
...,...,...,...,...,...
24,0.1,0.3,0.1,0.60,0.039217
23,0.1,0.3,0.1,0.40,0.039217
22,0.1,0.3,0.1,0.05,0.039217
40,0.2,0.2,0.1,0.05,0.039217


In [ ]:
eval(model, device, data_path)

1.00175702811245